In [1]:
import sys
!{sys.executable} -m pip install spotipy

import spotipy
import config
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import re 
import pandas as pd




[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [8]:
def load_track_features(tracks,spotify):
    offset = 0
    track_data = []
    while offset < len(tracks):
        ids = []
        for i in range(offset,min(offset+100,len(tracks))):
            ids.append(tracks[i]['spotify_id'])
        current_run = spotify.audio_features(ids)
        track_data += current_run
        offset = offset + 100
    return track_data

def parse_playlist(playlist):
    track_data = []
    
    for item in playlist['items']:
        track = item['track']
        track_dict = {}
        track_dict['liked_at'] = item['added_at']
        track_dict['name'] = track['name']
        track_dict['spotify_id'] = track['id']
        track_dict['artists'] = ",".join(i["name"] for i in track['artists'])
        track_dict['popularity'] = track['popularity']
        #for simplicity we're going to use the genre of the first artist on the track
        track_dict['artist_id'] = track['artists'][0]['id']
        track_data.append(track_dict)

    return track_data


#genres seem to be missing from both the track object and album object, but they're present in
# the artist object.  We're going to use artist to look up the genres and then join them on the
# dataframe.  
def load_artist_genres(tracks,spotify):
    offset = 0
    step_size = 20
    artist_genres = {}
    
    while offset < len(tracks):
        ids = []
        for i in range(offset,min(offset+step_size,len(tracks))):
            ids.append(tracks[i]['artist_id'])
        current_run = spotify.artists(ids)
        for data in current_run['artists']:
    
            artist_genres[data['id']] = data['genres']
        offset = offset + step_size
    return artist_genres
    
                       

In [6]:
def load_liked_songs(spotify):

    tracks = []
    offset = 0
    
    while True:
        current_run = spotify.current_user_saved_tracks(offset=offset,limit=50)
        tracks.extend(parse_playlist(current_run))
        offset = len(current_run['items']) + offset
        #There's 50 tracks/page, so if we ever have less than 50 we can stop paging
        if len(current_run['items']) < 50:
            break
    
    
    track_data = load_track_features(tracks,spotify)
    artist_genres = load_artist_genres(tracks,spotify)
    
    names_frame = pd.DataFrame.from_records(tracks).set_index('spotify_id')
    names_frame['genres'] = names_frame['artist_id'].map(artist_genres)
    return pd.DataFrame(track_data).rename(columns={"id":"spotify_id"}).set_index('spotify_id').join(names_frame)

In [7]:
config.init()
spotify = config.get_auth()
results = load_liked_songs(spotify)
results.to_csv("../data/liked_songs.csv")